# Рекомендация тарифов

Исходные данные - датасет с праметрами использования объемов опций тарифного плана (количество звонков, время разговора, количество использованных SMS и объемов интернет-траффика) клиентов телекоммуникационной компании.
Необходимо построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не требуется.
Модель должна обладать максимально возможным значением *accuracy* (при этом не менее 0.75).
*Accuracy* должно быть проверено на тестовой выборке.

In [1]:
import pandas as pd 
import warnings

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 

from sklearn.model_selection import GridSearchCV

In [2]:
warnings.simplefilter('ignore')#отключение предупреждений

## Изучение предоставленных данных

Для изучения предоставленных данных прочитаем его, получим первые 10 строк таблицы, изучим общую информацию о таблице, получим данные о количестве пропусков, полных дубликатов. Выведем описательные статистики значений, содержащихся в файле, списки и количество уникальных значений столбцов.

In [3]:
df = pd.read_csv(r'C:\Users\lebeda\Yandex_Practicum\Yandex_projects\data_sets\prjkt_6(sp9)\users_behavior.csv') #чтение предоставленного файла с данными

In [4]:
df.head() #вывод первых десяти строк таблицы

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
df.info() #получение информации о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [6]:
df.isna().sum() #получение данных о пропущенных значениях

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [7]:
df.duplicated().sum() #получение данных о полных дубликатах по строкам.

0

In [8]:
df.describe(include='all') #получение данных описательных статистик

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [9]:
df.query('is_ultra == 1')['is_ultra'].count() #количество записей для тарифа 'ultra'

985

In [10]:
df.query('is_ultra == 0')['is_ultra'].count() #количество записей для тарифа 'smart'

2229

Согласно предложенному описанию данных каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Перечень столбцов таблицы включает:

   - сalls — количество звонков;
   - minutes — суммарная длительность звонков в минутах;
   - messages — количество sms-сообщений;
   - mb_used — израсходованный интернет-трафик в Мб;
   - is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Вывод**

Таблица включает 3214 строк и 5 столбцов с информацией о объемах оказанных услуг (количество звонков и их длительность, количество отправленных сообщений и израсходованном интернет-траффике, а также информацию об используемом пользователем тарифе). Тип данных во всех столбцах, за исключеним типа тарфа - вещественные числа (float), для идентификации тарифа использованы категоризованные данные целочисленного типа (int): "1" - тариф 'ultra', "0" - тариф 'smart'. Большая часть данных описывает тариф 'smart' (2229 строк), меньшая - тариф 'ultra'(985 строк). Пропуски и дубликаты отсутсвуют.

## Разделение данных на тренировочную, тестовую и валидационную выборки.

Для получения выборок, используемых для обучения, тестирования и валидации моделей из предложенного набора создадим обучающую, тестовую и валидационную выборки в соотношении 60%-20%-20%, соответственно. Для этого дважды применим метод train_test_split библиотеки sklearn. За первый "проход" поделим таблицу на обучающую выборку (60%) и объединенную выборку для валидации и тестировния (40%). За второй "проход" поделим объединенную выборку пополам на собственно валидационную и выборку для тестирования (по 20% от исходной таблицы). Для равномерного выделения данных по признаку класса (принадлежность к тарифу) используем параметр stratify для которого в качестве значений укажем столбец принадлежности к тарифу ультра 'is_ultra'.

In [11]:
df_train, df_test_valid = train_test_split(df, test_size=0.4, random_state=12345, stratify=df['is_ultra'])
# получение обучающей выборки и объекдиненной выборки для тестирования и валидации результатов

In [12]:
df_valid, df_test = train_test_split(df_test_valid, test_size=0.5, random_state=12345, stratify=df_test_valid['is_ultra'])
# получение отдельных выборок для тестирования и валидации

In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 2294 to 2134
Data columns (total 5 columns):
calls       1928 non-null float64
minutes     1928 non-null float64
messages    1928 non-null float64
mb_used     1928 non-null float64
is_ultra    1928 non-null int64
dtypes: float64(4), int64(1)
memory usage: 90.4 KB


In [14]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 78 to 2434
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [15]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 3113 to 691
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


**Вывод**

Для обучения модели и валидации результатов из исходной таблицы методом  train_test_split библиотеки sklearn получены три выборки: обучающая, тестовая и валидационная в соотношении 3/1/1.

## Исследование моделей классификации

Проведем исследование трех обучающих моделей задачи классификации: "решающее дерево", "случайный лес", логистическая регрессия. Получим для каждой из них оценку качества и выберем лучшую. Для "решающего дерева" и  "случайного леса" оценку качества будем производить с изменением гиперпараметров (глубины и колличества оценщиков соответсвенно), определив их значения при лучшей оценке качества.

### Определение признаков и целевых признаков созданных выборок для обучения и валидации моделей

In [16]:
features_train = df_train.drop(['is_ultra'], axis=1) #определение признаков обучающей выборки
target_train = df_train['is_ultra'] #определение целевых признаков обучающей выборки

features_valid = df_valid.drop(['is_ultra'], axis=1) #определение признаков валидационной выборки
target_valid = df_valid['is_ultra'] #определение целевых признаков валидационной выборки

features_test = df_test.drop(['is_ultra'], axis=1) #определение признаков тестовой выборки
target_test = df_test['is_ultra'] #определение целевых признаков тестовой выборки

###  Исследование модели решающего дерева

In [17]:
# grid_params = {'max_depth': range(1,100,1)} #подбор гиперпараметров по сетке

# grid = GridSearchCV(DecisionTreeClassifier(random_state=12345),param_grid=grid_params,cv=3,scoring='accuracy')
# model_grid = grid.fit(features_valid, target_valid)
# print('max_model_score: '+str(model_grid.best_score_), str(model_grid.best_params_))

In [18]:
max_accuracy = 0
best_depth = 0
for depth in range(1, 100):
    model_1 = DecisionTreeClassifier(random_state=12345, max_depth=depth) # создание моднли решающего дерева с заданной глубиной дерева
    model_1.fit(features_train, target_train) # обучение модели
    predictions_valid = model_1.predict(features_valid) # предсказание на валидационной выборке 
    result = accuracy_score(target_valid, predictions_valid) # получение оценки качества модели
    if result > max_accuracy:
        max_accuracy = result
        best_depth = depth
        best_model_1 = model_1
        
print('max_accuracy:', max_accuracy, 'for depth:', best_depth)

max_accuracy: 0.80248833592535 for depth: 8


###  Исследование модели случайного леса

In [19]:
# grid_params = {'max_depth': range(1,50,5),     #подбор гиперпараметров по сетке
#                'n_estimators': range(1,100,5)}

# grid = GridSearchCV(RandomForestClassifier(random_state=12345),param_grid=grid_params,cv=3,scoring='accuracy')
# model_grid = grid.fit(features_valid, target_valid)
# print('max_model_score: '+str(model_grid.best_score_), str(model_grid.best_params_))

In [20]:
max_model_score = 0 
best_n_estimators = 0
best_depth = 0
for est in range(1, 100):
    for depth in range(1, 50):
        model_2 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) # создание моднли решающего дерева с заданной глубиной дерева
        model_2.fit(features_train, target_train) # обучение модели
        predictions_valid = model_2.predict(features_valid)
        result = model_2.score(features_valid, target_valid) # получение оценки качества модели
        if result > max_model_score:
            max_model_score = result
            best_n_estimators = est
            best_depth = depth
            best_model_2 = model_2
            
print('max_model_score:', max_model_score, 'for n_estimators:', best_n_estimators, 'and depth:', best_depth)

max_model_score: 0.8227060653188181 for n_estimators: 39 and depth: 9


###  Исследование модели логистической регрессии

In [21]:
model_3 = LogisticRegression(random_state=12345)
model_3.fit(features_train, target_train)
result = model_3.score(features_valid, target_valid) 

print('model_score:', result)

model_score: 0.71850699844479


### Вывод

Как видно из полученных данных самой высокой оценкой качества обладает модель "случайного леса" при количестве деревьев 39 с глубиной 9, далее по убыванию "решающее дерево" с глубиной 8 и логистическая регрессия. Следует также отметить, что время работы (обучения) у модели "случайного леса" с 58 деревьями самое большое. Скорость обучения модели "решающего дерева" глубиной 8 выше, при этом оценка качества достаточна близка к оценке качества "случайного леса". Самой простой и быстрой, но наименее точной оказалась логистическая регрессия.

## Проверка моделей на тестовой выборке

Проверим качество полученных моделей с оптимальными значениями гиперпараметров на тестовой выборке.

###  Проверка модели решающего дерева

In [22]:
predictions_test_1 = best_model_1.predict(features_test) # проверка модели решающего дерева с глубиной 3
result_test_1 = accuracy_score(target_test, predictions_test_1)
result_test_1

0.7838258164852255

###  Проверка модели случайного леса

In [23]:
###  Проверка модели решающего дерева# проверка модели случайного леса с колличеством деревьв 23
result_test_2 = best_model_2.score(features_test, target_test) 
result_test_2

0.8118195956454122

###  Проверка модели логистической регрессии

In [24]:
# проверка модели логистической регрессии
result_test_3 = model_3.score(features_test, target_test)
result_test_3

0.7107309486780715

### Вывод

Результат проверки моделей с оптимальными гиперпараметрами на тестовой выборке аналогичен результату работы на валидационной. Самым высоким качеством такжже обладает модель "случайного леса", далее по убыванию - "решающее дерево" и логистическая регрессия.

## Проверка моделей на адекватность

Для проверки моделей на адекватность будем использовать фиктивный классификатор данных DummyClassifier, обученный на тренировочной выборке. В качестве стратегии классификатора будем использовать стратегии 'most_frequent', 'prior', 'stratified', 'uniform'. Скорее всего для исследуемых данных самой эффиктивной стратегией фиктивного классификатора окажется 'most_frequent', старающуюся предсказать самый часто встречаюющийся тип класса в данных, а в исследуемых данных есть перекос по классам в одну сторону. Сравним оценку работы модели фиктивного классификатора (с учетом различных стратегий) с обученными моделями. Если оценка предсказаний фиктивного классиикатора ниже, чем оценка обученных моделей, можно считать, что обученные модели адекватны и работают лучше.

### Сравнение качества моделей с качеством модели фиктивного классификатора

In [25]:
strategy = ['most_frequent', 'prior', 'stratified', 'uniform']
for s in strategy:
    dummy_clf = DummyClassifier(strategy = s)
    dummy_clf.fit(features_train, target_train)
    predictions_dummy = dummy_clf.predict(features_test)
    result_dummy = accuracy_score(target_test, predictions_dummy)
    print ('dummy_accyracy:', result_dummy, ',',  'strategy:', s)
    if result_dummy < result_test_1:
        print('The 1st model is adequate')
    else:
        print('The 1st model is not adequate')
    if result_dummy < result_test_2:
        print('The 2st model is adequate')
    else:
        print('The 2st model is not adequate')
    if result_dummy < result_test_3:
        print('The 3st model is adequate')
    else:
        print('The 3st model is not adequate')
    print()

dummy_accyracy: 0.6936236391912908 , strategy: most_frequent
The 1st model is adequate
The 2st model is adequate
The 3st model is adequate

dummy_accyracy: 0.6936236391912908 , strategy: prior
The 1st model is adequate
The 2st model is adequate
The 3st model is adequate

dummy_accyracy: 0.5800933125972006 , strategy: stratified
The 1st model is adequate
The 2st model is adequate
The 3st model is adequate

dummy_accyracy: 0.48833592534992226 , strategy: uniform
The 1st model is adequate
The 2st model is adequate
The 3st model is adequate



### Вывод

Как видно из полученных данных, все исследуемые и обученные модели достаточно адекватны, оценка качества их работы выше, чем оценка качества работы фиктивного классификатора при любой из выбранных стратегий.

## Общий вывод

База исследований - файл формата .csv, содержащий информацию о объемах оказанных услуг мобильной связи: количество звонков и их длительность, количество отправленных сообщений и израсходованном интернет-траффике, а также информацию об используемом пользователем тарифе. 

Цель наити наиболее качественную модель обучения для задачи классификации с  максимально большим значением *accuracy*, которая выберет подходящий тариф.

В ходе работы ислледуемая таблица была поделена на выборки: тренировочную, валидационную и тестовую в соотношении 3/1/1. В качестве исследуемых моделей были выбраны: "решаюшее дерево", "случайный лес" и логистическая регрессия.

Были найдены наиболее оптимальные значения гиперпараметров модели для получения наибольших оценок качества (на интервале итерациий от 1 до 100): для "решающего дерева" - глубина 8, для "случайного леса" - 39 деревьев с глубиной 9. Наилучшим качетвом обладает модель "случайного леса", далее по убыванию - "решающее дерево" и логистическая регрессия.

Эффективность и качество работы моделей были проверены на тестовой выборке.

Была проверена и доказана адекватность моделей в сравнении с качеством работы фиктивного классификатора данных DummyClassifier с использованием разных его стратегий.